In [1]:
import sys
sys.path.append('..')
from ct.data.cub import CUB
from ct.model.cub_model import CUB_CT
import warnings
import argparse
import torch
import datetime
import pytorch_lightning as pl
import re
import os
import numpy as np
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)
from ct.model.cub_backbone import VIT_Backbone
class args:
    batch_size = 5
    epochs = 50
    lr = 1e-4
    dim = 1024
    num_heads = 16
    expl_coeff = 1
    n_global_concepts = 0
    n_spatial_concepts = 0
    n_classes = 0
dataset = CUB(args.batch_size)
dataset.setup()
args.n_global_concepts = dataset.n_global_attr
args.n_spatial_concepts = dataset.n_spatial_attr
args.n_classes = int(dataset.n_labels)


/home/ali/advDeep/dd2412-project/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset folder found. In order to download it, delete the existing folder!
Dataset folder found. In order to download it, delete the existing folder!


In [2]:
model_path = './logs/cub/2022-12-15_012505-bs=16,d=1024,e=200,ec=1.0,l=0.0001,nh=16,s=none,we=20'
model_path = os.path.join(model_path, 'checkpoint', 'epoch=119-step=40560.ckpt')
model = CUB_CT.load_from_checkpoint(model_path, args=args)

In [3]:
import pandas as pd
spatial_idx2attr_id = {}
for key, value in dataset.test.attr_id2local_id.items():
    spatial_idx2attr_id[value] = key
global_idx2attr_id = {}
for key, value in dataset.test.attr_id2global_id.items():
    global_idx2attr_id[value] = key
attributes = pd.read_csv('../data/cub/CUB_200_2011/attributes/attributes.txt', sep=' ', names=['attr_id', 'defi'])
attr_id2def = pd.Series(data=attributes.defi, index=attributes.attr_id).to_dict()

In [ ]:
device = 'cuda'
image_folder = '../data/cub/CUB_200_2011/images/'
model = model.to(device)
mu = [0.485, 0.456, 0.406]
sigma = [0.229, 0.224, 0.225]
display_corrects = True
max_batches = 5
for batch in dataset.test_dataloader():
    for k in batch.keys():
        if k == 'image_path': continue
        batch[k] = batch[k].to(device)
    batch['image'] = torch.autograd.Variable(batch['image'].data, requires_grad=True)
    target_class, global_concepts, spatial_concepts, predictions, attn_global, attn_spatial = model.predict_step(batch, None)
    for i in range(args.batch_size):
        if (predictions[i]==batch['label'][i]) ^ display_corrects:
            continue
        print(predictions[i], batch['label'][i])
        image = batch['image'][i].cpu().detach().numpy()
        image = np.moveaxis(image, 0, -1)
        plt.imshow(image)
        plt.show()
        global_attn = attn_global[i][0].cpu().detach().numpy()
        spatial_attn = attn_spatial[i].cpu().detach().numpy()   
        print(batch['image_path'][0][i])
        for idx, a in enumerate(global_attn):
            if a > 0:
                print(attr_id2def[global_idx2attr_id[idx]])
        for a in spatial_attn:
            if a.max() > 0:
                print(attr_id2def[spatial_idx2attr_id[np.argmax(a)]])
        image[0] = image[0]*sigma[0] + mu[0]
        image[1] = image[1]*sigma[1] + mu[1]
        image[2] = image[2]*sigma[2] + mu[2]
        break
    max_batches -= 1
    if max_batches <= 0:
        break